In [ ]:
# WARNING: This experiment relies on the content of the data folder
#          pointed to by 'yaml_mixed'.
# Original Run 1: Data folder contained only 'Landscape' images.
# Original Run 2: Data folder contained 'Landscape' + 'Virtual' images.
#
# Future users must ensure the data folder is configured for the intended run.
# Current execution assumes the FULL mixed dataset is present.

Installing ultralytics package

In [ ]:
#!pip install ultralytics
#!pip install --upgrade ultralytics

@misc{ultralytics2023yolov8,
  title={{YOLOv8: YOLO by Ultralytics, Version 8.0}},
  author={Ultralytics},
  year={2023},
  publisher={GitHub},
  howpublished={\url{https://github.com/ultralytics/ultralytics}},
  note={Accessed: [Date Accessed]}
}

Import Section

In [ ]:
from ultralytics import YOLO
from datetime import datetime
import os

## Train on virtual ( Landcsape ) and test on real

In [ ]:
# List of  YAML dataset configs
base_dir = r"C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\data"
experiments = [
    ("virtual-only", os.path.join(base_dir, "virtual.yaml"))
]

# In this case just one experiment, change as prefer
for name, yaml_path in experiments:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    run_name = f"{name}_{timestamp}"
    print(f"\n Starting training: {name}\n")

    model = YOLO("yolov8n.pt")  # you can change to yolov8s.pt if you want a bit more accuracy
    model.train(
        data=yaml_path,
        epochs=50,
        imgsz=640,
        batch=16,
        project="runs_yolo",
        name=run_name,
        seed=42,
        val=True,          # enable validation after each epoch
        exist_ok=True,     
    )

print("\n All experiments completed!")


## Train on real, mixed and test on real 

In [ ]:
# Other experiments with different set ups, training data are real_only and mixed-50-50
#  ( in this set up the mixed_LRDD dataset already contains city images due to a subsequent addition )
# We suggest recreating the mixed LRDD dataset to exclude city images
base_dir = r"C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\data"
experiments = [
    
    ("real-only", os.path.join(base_dir, "real_LRDD.yaml")),
    ("mixed-50-50", os.path.join(base_dir, "mixed_LRDD.yaml"))
]

# Loop through experiments
for name, yaml_path in experiments:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    run_name = f"{name}_{timestamp}"
    print(f"\n Starting training: {name}\n")

    model = YOLO("yolov8n.pt")  # you can change to yolov8s.pt if you want a bit more accuracy
    model.train(
        data=yaml_path,
        epochs=50,
        imgsz=640,
        batch=16,
        project="runs_yolo",
        name=run_name,
        seed=42,
        val=True,          # enable validation after each epoch
        exist_ok=True,     # overwrite folder if re-running
    )

print("\nAll experiments completed!")


## 50/50 Mixed with city images

In [ ]:
# === CONFIG ===
base_dir = r"C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\data"
yaml_path = os.path.join(base_dir, "mixed_LRDD.yaml")

# === TRAINING ===
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
run_name = f"mixed_50_50_withcity{timestamp}"

print(f"\n Starting YOLO training on mixed dataset (50/50)\n")

model = YOLO("yolov8n.pt")  # or yolov8s.pt for better accuracy

model.train(
    data=yaml_path,
    epochs=50,
    imgsz=640,
    batch=16,
    project="runs_yolo",
    name=run_name,
    seed=42,
    val=True,          # validate every epoch
    exist_ok=True,     # overwrite existing folder
)

print(f"\n Training completed for: {run_name}")


### Now we try with the 70/30 datasets in different variations 

These experiments were performed before the addition of city images

In [ ]:
# === COMMON SETTINGS (fits 4GB GPU) ===
COMMON = dict(
    epochs=50,
    imgsz=640,
    batch=4,          # it was 8
    workers=2,
    seed=42,
    project="runs_yolo",
)

# === PATHS ===
base_dir = r"C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\data"

# YAML files
yaml_mixed_70_30 = os.path.join(base_dir, "mixed_LRDD_70_30.yaml")
yaml_virtual = os.path.join(base_dir, "virtual.yaml")
yaml_real = os.path.join(base_dir, "real_LRDD.yaml")

# === EXPERIMENT 1: Baseline 70/30 Mixed ===
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
print(f"\n Starting Experiment 1: 70/30 Mixed Baseline ({timestamp})\n")

model = YOLO("yolov8n.pt")
model.train(
    data=yaml_mixed_70_30,
    name=f"mixed_70_30_baseline_{timestamp}",
    **COMMON
)


### Pretrain Virtual and Finetune real

In [ ]:
# === EXPERIMENT 2: Curriculum (Pretrain Virtual -> Finetune Real) ===
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
print(f"\n Starting Experiment 2: Curriculum (Virtual → Real) ({timestamp})\n")

# Stage A - Pretrain on virtual (short warm-up)
model = YOLO("yolov8n.pt")
model.train(
    data=yaml_virtual,
    epochs=15,
    imgsz=640,
    batch=4,
    workers=2,
    project="runs_yolo",
    name=f"curriculum_stageA_virtual_{timestamp}",
)
# Stage B - Fine-tune on real
model = YOLO(
    fr"C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\notebooks\runs_yolo\curriculum_stageA_virtual_{timestamp}\weights\best.pt"

)

model.train(
    data=yaml_real,
    lr0=0.002,     # smaller LR to preserve learned features
    freeze=10,     # optional: freeze early layers initially
    name=f"curriculum_stageB_real_{timestamp}",
    **COMMON
)




### Strong Augmentation Case


In [ ]:
# === EXPERIMENT 3: 70/30 Mixed + Strong Augmentation ===
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
print(f"\n Starting Experiment 3: 70/30 Mixed with Strong Augmentation ({timestamp})\n")

model = YOLO("yolov8n.pt")
model.train(
    data=yaml_mixed_70_30,
    name=f"mixed_70_30_augmented_{timestamp}",
    hsv_h=0.03, hsv_s=0.9, hsv_v=0.6,
    erasing=0.6, translate=0.2, scale=0.6, shear=2.0, perspective=0.0005,
    mosaic=0.7, close_mosaic=10,
    **COMMON
)

print("\n All experiments completed successfully!\n")